# user0 - создание холодного пользователя

In [2]:
import pandas as pd

# Создаем DataFrame с тегами
tags = [
    'Мультфильмы', 'Развлечения', 'Хобби', 'Животные', 'Разное', 
    'Лайфстайл', 'Телепередачи', 'Спорт', 'Недвижимость', 'Сериалы', 
    'Детям', 'Музыка', 'Юмор', 'Авто-мото', 'Сад и огород', 
    'Фильмы', 'Видеоигры', 'Наука', 'Путешествия', 'Технологии и интернет', 
    'Лайфхаки', 'Культура', 'Обзоры и распаковки товаров', 'Обучение', 
    'Здоровье', 'Охота и рыбалка', 'Строительство и ремонт', 'Аниме', 
    'Эзотерика', 'Еда', 'UNKNOWN', 'Интервью', 'Бизнес и предпринимательство', 
    'Техника и оборудование', 'Психология', 'Красота', 'Природа', 
    'Дизайн', 'Аудиокниги', 'Аудио'
]

# Создаем DataFrame
df = pd.DataFrame({
    'tag': tags,
    'percent': [0.000] * len(tags),
    'likes': [0] * len(tags)
})

# Сохраняем DataFrame в файл формата Parquet
df.to_parquet('user0_tags.parquet', index=False)

In [3]:
import pandas as pd

# Загружаем DataFrame из файла Parquet
df = pd.read_parquet('user0_tags.parquet')

# Выводим первые 5 строк
print(df.head())

           tag  percent  likes
0  Мультфильмы      0.0      0
1  Развлечения      0.0      0
2        Хобби      0.0      0
3     Животные      0.0      0
4       Разное      0.0      0


In [8]:
import pandas as pd

# Загрузка файла
df = pd.read_parquet('logsV02.parquet')

# Получение всех уникальных значений в столбце region
unique_regions = df['city'].unique()

# Создание нового DataFrame с уникальными регионами и заданными значениями
data = {
    'city': unique_regions,
    'percent': [0.0] * len(unique_regions),
    'likes': [0] * len(unique_regions)
}

df_unique = pd.DataFrame(data)

# Сохранение DataFrame в файл формата Parquet
df_unique.to_parquet('user0_city.parquet', index=False)

print("Данные успешно сохранены в user0_city.parquet")

Данные успешно сохранены в user0_city.parquet


In [17]:
import pandas as pd

# Загружаем DataFrame из файла Parquet
df = pd.read_parquet('users/user0_city.parquet')

# Выводим первые 5 строк
print(df.head(-5))

                                     city  percent  likes
0    daa223d9-5a07-4723-8b1c-e0660fb50036      0.0      0
1    d228cf4d-daae-4a4c-b9e0-aa64d8c54352      0.0      0
2    c8910e58-e5da-4ba9-bb99-1c8152ef2942      0.0      0
3    633d9f2d-7453-417d-8454-0ce9f935b693      0.0      0
4    62453693-e6de-4ff7-a06c-1ba765266f35      0.0      0
..                                    ...      ...    ...
802  64e311c2-5870-40cb-9007-c817992cfee6      0.0      0
803  94182336-c2c3-4755-8190-c706780b96d3      0.0      0
804  c1a311b1-955f-4e93-9fd7-666bf6c7a39e      0.0      0
805  1c5b06bb-a8dc-4e17-acf1-83379eddebfd      0.0      0
806  e6e4ac68-a49c-4cd4-b86c-4d4ada9c5104      0.0      0

[807 rows x 3 columns]


# Сбор

## Tags. Сюда приходят значения лайков, дизлайков, после обновления страницы

In [1]:
import pandas as pd

# Загрузка данных из файлов
tags_df = pd.read_parquet('users/user0_tags.parquet')
categories_df = pd.read_parquet('tags-filtered.parquet')

# Данные на входе
aggregated = [
    {'uid': "6ba4b910-2b84-462e-92c7-477d29df3da0", 'total_likes': 1, 'total_dislikes': 0},
    {'uid': "a766398d-ae80-4e6d-8da6-dc306903f8b0", 'total_likes': 0, 'total_dislikes': 1},
    {'uid': "9d000f21-5760-481c-9b66-552dd28551a6", 'total_likes': 0, 'total_dislikes': 0},
]

# Инициализация словаря для накопления лайков и дизлайков по категориям
category_likes = {}

# Суммирование лайков и дизлайков для каждой категории
for item in aggregated:
    video_id = item['uid']
    
    # Получение category_id по video_id
    category_info = categories_df[categories_df['video_id'] == video_id]
    
    if not category_info.empty:
        category_id = category_info['category_id'].values[0]
        
        if category_id not in category_likes:
            category_likes[category_id] = {'likes': 0, 'dislikes': 0}
        
        category_likes[category_id]['likes'] += item['total_likes']
        category_likes[category_id]['dislikes'] += item['total_dislikes']

# Обновление тегов на основе накопленных значений
for category_id, counts in category_likes.items():
    tag_index = tags_df[tags_df['tag'] == category_id].index
    
    if not tag_index.empty:
        tag_row = tag_index[0]
        
        # Обновляем.likes в tags_df
        tags_df.at[tag_row, 'likes'] += counts['likes']
        tags_df.at[tag_row, 'likes'] = max(0, tags_df.at[tag_row, 'likes'] - counts['dislikes'])

# Функция для расчета процентов
def calculate_percentages(df):
    total_likes = df['likes'].sum()
    if total_likes > 0:
        df['percent'] = (df['likes'] / total_likes * 100).fillna(0)
    else:
        df['percent'] = 0
    return df

# Расчет лайков и процентов для тегов
tags_df = calculate_percentages(tags_df)

# Сохранение обновленного DataFrame обратно в файл
tags_df.to_parquet('users/user0_tags.parquet', index=False)

print("Лайки и процентное соотношение сохранены в файл users/user0_tags.parquet.")

Лайки и процентное соотношение сохранены в файл users/user0_tags.parquet.


In [58]:
import pandas as pd

# Загружаем DataFrame из файла Parquet
df = pd.read_parquet('users/user0_tags.parquet')

# Выводим первые 5 строк
print(df.head(200))

                             tag    percent  likes
0                    Мультфильмы   0.000000      0
1                    Развлечения  66.666667      4
2                          Хобби   0.000000      0
3                       Животные   0.000000      0
4                         Разное   0.000000      0
5                      Лайфстайл   0.000000      0
6                   Телепередачи   0.000000      0
7                          Спорт   0.000000      0
8                   Недвижимость   0.000000      0
9                        Сериалы   0.000000      0
10                         Детям   0.000000      0
11                        Музыка   0.000000      0
12                          Юмор   0.000000      0
13                     Авто-мото   0.000000      0
14                  Сад и огород   0.000000      0
15                        Фильмы  33.333333      2
16                     Видеоигры   0.000000      0
17                         Наука   0.000000      0
18                   Путешестви

## City

In [76]:
import pandas as pd

# Загрузка данных из файла cities
cities_df = pd.read_parquet('users/user0_regions.parquet')
logs_df = pd.read_parquet('logsV02.parquet')

# Данные на входе
aggregated = [
    {'uid': "6ba4b910-2b84-462e-92c7-477d29df3da0", 'total_likes': 0, 'total_dislikes': 0},
    {'uid': "a766398d-ae80-4e6d-8da6-dc306903f8b0", 'total_likes': 0, 'total_dislikes': 1},
    {'uid': "9d000f21-5760-481c-9b66-552dd28551a6", 'total_likes': 1, 'total_dislikes': 0},
]

# Функция для обновления лайков и подсчета процентов
def update_city_likes(cities_df, item):
    video_id = item['uid']
    
    # Получаем все записи по video_id
    log_entries = logs_df[logs_df['video_id'] == video_id]
    
    if not log_entries.empty:
        # Находим регион с максимальным значением watchtime
        city_with_max_watchtime = log_entries.loc[log_entries['watchtime'].idxmax(), 'region']
        print(city_with_max_watchtime)

        # Обновляем количество лайков для найденного города
        update_likes(cities_df, {'total_likes': item['total_likes'], 'total_dislikes': item['total_dislikes']}, city_with_max_watchtime)

# Функция для обновления лайков в DataFrame
def update_likes(df, item, key):
    index = df[df['region'] == key].index
    if not index.empty:
        row = index[0]
        current_likes = df.at[row, 'likes']

        # Увеличиваем или уменьшаем количество лайков в зависимости от состояния
        if item['total_likes'] > 0:
            df.at[row, 'likes'] += item['total_likes']
        if item['total_dislikes'] > 0:
            if current_likes > 0:
                df.at[row, 'likes'] = max(0, current_likes - item['total_dislikes'])

# Обновление лайков для каждого элемента в aggregated
for item in aggregated:
    update_city_likes(cities_df, item)

# Функция для расчета процентов
def calculate_percentages(df):
    total_likes = df['likes'].sum()
    df['percent'] = (df['likes'] / total_likes * 100).fillna(0)
    return df

# Расчет процентов для городов
cities_df = calculate_percentages(cities_df)

# Сохранение обновленных DataFrame обратно в файл
cities_df.to_parquet('users/user0_city.parquet', index=False)

print("Лайки и процентное соотношение сохранены в файл users/user0_region.parquet.")

58a3fdfe-957e-48de-a05b-6d7fd62ae0b2
1252b9d0-d457-4967-b0a8-c3b3fc064630
4496e8c2-fe04-4654-995c-4e39173f0a77
Лайки и процентное соотношение сохранены в файл users/user0_region.parquet.


In [77]:
import pandas as pd

# Загружаем DataFrame из файла Parquet
df = pd.read_parquet('users/user0_city.parquet')

# Выводим первые 5 строк
print(df.head(200))

                                   region    percent  likes
0    d0a727cb-ac6b-4bf7-8449-73a5773ee817   0.000000      0
1    1252b9d0-d457-4967-b0a8-c3b3fc064630   0.000000      0
2    5e72a4fe-48c1-4b6c-9b85-078f490357dd   0.000000      0
3    57ca7ea4-0a03-465d-9302-6ec6b350eefd   0.000000      0
4    4496e8c2-fe04-4654-995c-4e39173f0a77  38.461538      5
..                                    ...        ...    ...
195  03c5c9f1-43bc-4851-981c-37ef2999076f   0.000000      0
196  c4226b87-c62b-4a5f-b600-5b66cbd6c42a   0.000000      0
197  13042133-f2cf-4caa-94de-b6275fd64f80   0.000000      0
198  a40cc51b-ffa8-4472-a4e2-12ebf765b30e   0.000000      0
199  f4da1530-1ae8-40ab-9ab5-5a3ecf3213d8   0.000000      0

[200 rows x 3 columns]


## Region

In [78]:
import pandas as pd

# Загрузка данных из файла cities
cities_df = pd.read_parquet('users/user0_regions.parquet')
logs_df = pd.read_parquet('logsV02.parquet')

# Данные на входе
aggregated = [
    {'uid': "6ba4b910-2b84-462e-92c7-477d29df3da0", 'total_likes': 1, 'total_dislikes': 0},
    {'uid': "a766398d-ae80-4e6d-8da6-dc306903f8b0", 'total_likes': 0, 'total_dislikes': 1},
    {'uid': "9d000f21-5760-481c-9b66-552dd28551a6", 'total_likes': 0, 'total_dislikes': 0},
]

# Функция для обновления лайков и подсчета процентов
def update_city_likes(cities_df, item):
    video_id = item['uid']
    
    # Получаем все записи по video_id
    log_entries = logs_df[logs_df['video_id'] == video_id]
    
    if not log_entries.empty:
        # Находим регион с максимальным значением watchtime
        city_with_max_watchtime = log_entries.loc[log_entries['watchtime'].idxmax(), 'region']
        print(city_with_max_watchtime)

        # Обновляем количество лайков для найденного города
        update_likes(cities_df, {'total_likes': item['total_likes'], 'total_dislikes': item['total_dislikes']}, city_with_max_watchtime)

# Функция для обновления лайков в DataFrame
def update_likes(df, item, key):
    index = df[df['region'] == key].index
    if not index.empty:
        row = index[0]
        current_likes = df.at[row, 'likes']

        # Увеличиваем или уменьшаем количество лайков в зависимости от состояния
        if item['total_likes'] > 0:
            df.at[row, 'likes'] += item['total_likes']
        if item['total_dislikes'] > 0:
            if current_likes > 0:
                df.at[row, 'likes'] = max(0, current_likes - item['total_dislikes'])

# Обновление лайков для каждого элемента в aggregated
for item in aggregated:
    update_city_likes(cities_df, item)

# Функция для расчета процентов
def calculate_percentages(df):
    total_likes = df['likes'].sum()
    df['percent'] = (df['likes'] / total_likes * 100).fillna(0)
    return df

# Расчет процентов для городов
cities_df = calculate_percentages(cities_df)

# Сохранение обновленных DataFrame обратно в файл
cities_df.to_parquet('users/user0_regions.parquet', index=False)

print("Лайки и процентное соотношение сохранены в файл users/user0_region.parquet.")

58a3fdfe-957e-48de-a05b-6d7fd62ae0b2
1252b9d0-d457-4967-b0a8-c3b3fc064630
4496e8c2-fe04-4654-995c-4e39173f0a77
Лайки и процентное соотношение сохранены в файл users/user0_region.parquet.


In [73]:
import pandas as pd

# Загружаем DataFrame из файла Parquet
df = pd.read_parquet('users/user0_regions.parquet')

# Выводим первые 5 строк
print(df.head(200))

                                   region    percent  likes
0    d0a727cb-ac6b-4bf7-8449-73a5773ee817   0.000000      0
1    1252b9d0-d457-4967-b0a8-c3b3fc064630   0.000000      0
2    5e72a4fe-48c1-4b6c-9b85-078f490357dd   0.000000      0
3    57ca7ea4-0a03-465d-9302-6ec6b350eefd   0.000000      0
4    4496e8c2-fe04-4654-995c-4e39173f0a77  33.333333      4
..                                    ...        ...    ...
195  03c5c9f1-43bc-4851-981c-37ef2999076f   0.000000      0
196  c4226b87-c62b-4a5f-b600-5b66cbd6c42a   0.000000      0
197  13042133-f2cf-4caa-94de-b6275fd64f80   0.000000      0
198  a40cc51b-ffa8-4472-a4e2-12ebf765b30e   0.000000      0
199  f4da1530-1ae8-40ab-9ab5-5a3ecf3213d8   0.000000      0

[200 rows x 3 columns]


###### Поиск

In [10]:
import pandas as pd

# Загрузка данных из файла
#cities_df = pd.read_parquet('users/user0_city.parquet')
cities_df = pd.read_parquet('logsV02.parquet')
#6ba4b910-2b84-462e-92c7-477d29df3da0 - video_id
# Определенный video_id для поиска
search_video_id = "d228cf4d-daae-4a4c-b9e0-aa64d8c54352"  # замените на нужный video_id

# Фильтрация DataFrame по video_id
result = cities_df[cities_df['city'] == search_video_id]

# Проверка результата
if not result.empty:
    print("Найдены значения для video_id:", search_video_id)
    print(result)  # выводим найденные строки
else:
    print("Не найдены значения для video_id:", search_video_id)

Найдены значения для video_id: d228cf4d-daae-4a4c-b9e0-aa64d8c54352
                                    region  \
1     1252b9d0-d457-4967-b0a8-c3b3fc064630   
416   1252b9d0-d457-4967-b0a8-c3b3fc064630   
919   1252b9d0-d457-4967-b0a8-c3b3fc064630   
2395  1252b9d0-d457-4967-b0a8-c3b3fc064630   

                                      city  \
1     d228cf4d-daae-4a4c-b9e0-aa64d8c54352   
416   d228cf4d-daae-4a4c-b9e0-aa64d8c54352   
919   d228cf4d-daae-4a4c-b9e0-aa64d8c54352   
2395  d228cf4d-daae-4a4c-b9e0-aa64d8c54352   

                                  video_id  watchtime  
1     a766398d-ae80-4e6d-8da6-dc306903f8b0        673  
416   87fbbc1b-0379-4e27-a219-3594fe0f0359         31  
919   788b1234-da5c-43d4-b316-4742d3c47d97          0  
2395  a83f11be-95d5-4b92-bb1c-a9ffe047a8a8       1563  
